# FFNet (Hartford et al 2016)

## Session

### Start

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
sess = tf.InteractiveSession()

## Data

### Augmentation

In [2]:
p = [[[1,0,0],[0,1,0],[0,0,1]],[[1,0,0],[0,0,1],[0,1,0]],
     [[0,1,0],[1,0,0],[0,0,1]],[[0,1,0],[0,0,1],[1,0,0]],
     [[0,0,1],[1,0,0],[0,1,0]],[[0,0,1],[0,1,0],[1,0,0]]]

### No augmentation

In [3]:
p = [[[1,0,0],[0,1,0],[0,0,1]],[[1,0,0],[0,1,0],[0,0,1]],
     [[1,0,0],[0,1,0],[0,0,1]],[[1,0,0],[0,1,0],[0,0,1]],
     [[1,0,0],[0,1,0],[0,0,1]],[[1,0,0],[0,1,0],[0,0,1]]]

### By rows

In [4]:
import_csv = pd.read_csv("games3x3.csv")
inputs_set = np.zeros((import_csv.shape[0]*6,18))
target_set = np.zeros((import_csv.shape[0]*6,3))
for i in range(import_csv.shape[0]*6)[::6]:
    a = np.matrix(import_csv['matrix'][i/6])
    a = (a-np.mean(a))/np.std(a)
    for j in range(6):
        b = a*p[j]
        c = np.array(b).flatten()
        d = np.transpose(a)
        e = d*p[j]
        f = np.array(e).flatten()
        inputs_set[i+j] = np.concatenate((c, f), axis=0)
        target_set[i+j] = np.matrix(import_csv['choice'][i/6])

### By rows and columns

In [5]:
for k in range(1,6):
    inputs_tmp = np.zeros((import_csv.shape[0]*6,18))
    target_tmp = np.zeros((import_csv.shape[0]*6,3))
    for i in range(import_csv.shape[0]*6)[::6]:
        a = p[k]*np.matrix(import_csv['matrix'][i/6])
        a = (a-np.mean(a))/np.std(a)
        for j in range(6):
            b = a*p[j]
            c = np.array(b).flatten()
            d = np.transpose(a)
            e = d*p[j]
            f = np.array(e).flatten()
            inputs_tmp[i+j] = np.concatenate((c, f), axis=0)
            target_tmp[i+j] = np.transpose(p[k]*np.transpose(np.matrix(import_csv['choice'][i/6])))
    inputs_set = np.concatenate((inputs_set, inputs_tmp), axis=0)
    target_set = np.concatenate((target_set, target_tmp), axis=0)

### Training and test sets

In [6]:
shuffle_index_set = np.random.permutation(range(inputs_set.shape[0]))

In [7]:
train_index_set = shuffle_index_set[0:inputs_set.shape[0]/5*4] 
tests_index_set = shuffle_index_set[inputs_set.shape[0]/5*4:inputs_set.shape[0]]
train_inputs_set = inputs_set[train_index_set]
train_target_set = target_set[train_index_set]
tests_inputs_set = inputs_set[tests_index_set]
tests_target_set = target_set[tests_index_set]

## Model 

### Input and target layer

In [8]:
x = tf.placeholder(tf.float32, shape=[None, 18])
y_ = tf.placeholder(tf.float32, shape=[None, 3])

## Model

### Weights and bias

In [9]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

### Fully-connected layers

In [10]:
hi = 50

In [11]:
W_1 = weight_variable([18, hi])
b_1 = bias_variable([hi])
h_1 = tf.nn.relu(tf.matmul(x, W_1) + b_1)

In [12]:
W_2 = weight_variable([hi, hi])
b_2 = bias_variable([hi])
h_2 = tf.nn.relu(tf.matmul(h_1, W_2) + b_2)

### Dropout
- Drop probability = 0.2

In [13]:
keep_prob = tf.placeholder(tf.float32)
h_2_drop = tf.nn.dropout(h_2, keep_prob)

### Readout layer

In [14]:
W_3 = weight_variable([hi, 3])
b_3 = bias_variable([3])
y = tf.nn.softmax(tf.matmul(h_2_drop, W_3) + b_3)

### Cost function

#### $L_2$ regularization 
- $ \beta = 0.01 $

In [15]:
beta = 0.01
regularizer = tf.nn.l2_loss(W_1) + tf.nn.l2_loss(W_2) + tf.nn.l2_loss(W_3)
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, y_) + beta * regularizer)

### Step

#### Adam optimizer
- initial learning rate = $ 0.0002 $
- $ \beta_1 = 0.9 $
- $ \beta_2 = 0.999 $
- $ \epsilon = 1e-8 $

In [16]:
train_step = tf.train.AdamOptimizer(0.0002,0.9,0.999,1e-8).minimize(cross_entropy)

## Train

In [17]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())
for i in range(2500):
    shuffle_index_batch = np.random.permutation(range(train_inputs_set.shape[0]))[0:100]
    train_inputs_batch = train_inputs_set[shuffle_index_batch]
    train_target_batch = train_target_set[shuffle_index_batch]
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x: train_inputs_batch, y_: train_target_batch, keep_prob: 1.0})
        train_NLL = cross_entropy.eval(feed_dict={x: train_inputs_batch, y_: train_target_batch, keep_prob: 1.0})
        print("step %d, train accuracy %g, train NLL %g"%(i, train_accuracy, train_NLL))
    train_step.run(feed_dict={x: train_inputs_batch, y_: train_target_batch, keep_prob: 0.8})

step 0, train accuracy 0.51, train NLL 1.23575
step 100, train accuracy 0.54, train NLL 1.17412
step 200, train accuracy 0.72, train NLL 1.11571
step 300, train accuracy 0.71, train NLL 1.07422
step 400, train accuracy 0.77, train NLL 1.03909
step 500, train accuracy 0.85, train NLL 0.99669
step 600, train accuracy 0.91, train NLL 1.00545
step 700, train accuracy 0.9, train NLL 0.975917
step 800, train accuracy 0.96, train NLL 0.985765
step 900, train accuracy 0.98, train NLL 0.968614
step 1000, train accuracy 0.95, train NLL 1.00366
step 1100, train accuracy 0.94, train NLL 0.983638
step 1200, train accuracy 0.94, train NLL 0.984966
step 1300, train accuracy 0.98, train NLL 0.950184
step 1400, train accuracy 0.97, train NLL 0.950666
step 1500, train accuracy 0.92, train NLL 0.968138
step 1600, train accuracy 0.97, train NLL 0.981955
step 1700, train accuracy 0.97, train NLL 0.961901
step 1800, train accuracy 0.95, train NLL 0.968691
step 1900, train accuracy 0.91, train NLL 0.990188
s

## Evaluate

In [18]:
test_accuracy = accuracy.eval(feed_dict={x: tests_inputs_set, y_: tests_target_set, keep_prob: 1.0})
test_NLL = cross_entropy.eval(feed_dict={x: tests_inputs_set, y_: tests_target_set, keep_prob: 1.0})
print("test accuracy %g, test NLL %g"%(test_accuracy, test_NLL))

test accuracy 0.924797, test NLL 0.968673


In [19]:
y.eval(feed_dict={x: tests_inputs_set, y_: tests_target_set, keep_prob: 1.0})

array([[ 0.8980031 ,  0.01143877,  0.09055815],
       [ 0.55918431,  0.00892323,  0.43189245],
       [ 0.65178853,  0.01931819,  0.32889321],
       ..., 
       [ 0.95829874,  0.0060634 ,  0.03563783],
       [ 0.58665484,  0.33894208,  0.07440313],
       [ 0.86362469,  0.08052094,  0.05585435]], dtype=float32)

In [20]:
y_.eval(feed_dict={x: tests_inputs_set, y_: tests_target_set, keep_prob: 1.0})

array([[ 0.64999998,  0.04      ,  0.31      ],
       [ 0.47999999,  0.01333333,  0.50666666],
       [ 0.51127821,  0.18045112,  0.30827066],
       ..., 
       [ 0.84020621,  0.03092784,  0.12886597],
       [ 0.32333332,  0.49666667,  0.18000001],
       [ 0.75773197,  0.14948453,  0.0927835 ]], dtype=float32)